In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!nvidia-smi

Thu May  5 23:50:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.0 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 6.6 MB 41.7 MB/s 
     |████████████████████████████████| 596 kB 60.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
##importing all the packages

import os
import sys
import torch
from transformers import BertForTokenClassification
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from transformers import RobertaModel
from transformers import BertModel
from transformers import RobertaForSequenceClassification

import csv,pprint,datetime,random,time
import numpy as np
from sklearn import metrics


In [ ]:
###initializing the directories 

##Provide the home_dir ..rest will be taken care of!
home_dir = "gdrive/My Drive/project_5_data"
data_dir = os.path.join(home_dir, "datasets")
model_dir = os.path.join(home_dir, "model_dir")
if not os.path.isdir(model_dir):
  os.mkdir(model_dir)

Tagging_schema='BIOE'
epochs=5
batch_size=8

In [ ]:
##AS there is slight mismatch in the dataset test template and no of files given..Used to handle that issue

import pandas as pd


def clean_test_data():


  label_file = os.path.join(data_dir, "test-task-tc-template.out")
  myfile = open(label_file)
  prev_index = -1
  tsvreader = csv.reader(myfile, delimiter="\t")
  df = pd.DataFrame(tsvreader, index=None)
  # df.head()
  file_ids=df[0].to_list()
  file_ids=list(set(file_ids))
  print(len(file_ids))
  return file_ids

In [ ]:
def read_articles(article_dir):
  '''Function to read articles from the given folders '''

  articles = []

  article_dir = os.path.join(data_dir, article_dir)
  articles_files=sorted(os.listdir(article_dir))
  print(articles_files)
  for filename in articles_files:
    f = open(os.path.join(article_dir, filename))
    article = f.read()
    articles.append(article)
    f.close()

  article_ids = []
  article_fullnames=[]
  for filename in articles_files:
    
    tmp=filename[7:-4]
    article_ids.append(tmp)
    article_fullnames.append(filename)
  return articles, article_ids,article_fullnames

In [ ]:
#a1, a2,a3=read_articles('test-articles')

In [ ]:
#len(a3)

In [ ]:
# Read training span labels 
def read_spans(mode=None):
  spans,techniques = [],[]
  if mode == "test":
    label_dir = os.path.join(data_dir, "dev-labels-task-flc-tc")
  else:
    label_dir = os.path.join(data_dir, "train-labels-task-flc-tc")
  sorted_lst=sorted(os.listdir(label_dir))
  for filename in sorted_lst:
    f = open(os.path.join(label_dir, filename))
    filereader = csv.reader(f, delimiter="\t")
    span,technique = [],[]
    for row in filereader:
      span.append((int(row[2]), int(row[3])))
      if mode == "test":
        technique.append("Slogans")
      else:
        technique.append(row[1])
    f.close()
    spans.append(span)
    techniques.append(technique)
  return spans, techniques

In [ ]:
# Read dev span labels 
def read_dev_spans():
  spans,techniques = [],[]
  label_file = os.path.join(data_dir, "dev-task-flc-tc.labels")
  f = open(label_file)
  prev_index = -1. #just to make sure to know if the prevfile is same or not
  filereader = csv.reader(f, delimiter="\t")

  span,technique = [],[]
  for line in filereader:

    article_index = int(line[0])
    if article_index != prev_index:
      if prev_index != -1:
        spans.append(span)
        techniques.append(technique)
      span = []
      technique = []
      span.append((int(line[2]), int(line[3])))
      technique.append("Slogans")
      prev_index = article_index
    else:
      span.append((int(line[2]), int(line[3])))
      technique.append("Slogans")
  spans.append(span)
  techniques.append(technique)
  return spans, techniques, 

In [ ]:
# Read training span labels 
def read_test_spans():
  spans,techniques = [],[]
  label_file = os.path.join(data_dir, "test-task-tc-template.out")
  f = open(label_file)
  prev_index = -1
  filereader = csv.reader(f, delimiter="\t")

  span,technique = [],[]
  for line in filereader:
    article_index = int(line[0])
    if article_index != prev_index:
      if prev_index != -1:
        spans.append(span)
        techniques.append(technique)
      span = []
      technique = []
      span.append((int(line[2]), int(line[3])))
      technique.append("Slogans")
      prev_index = article_index
    else:
      span.append((int(line[2]), int(line[3])))
      technique.append("Slogans")
  spans.append(span)
  techniques.append(technique)
  return spans, techniques

In [ ]:
def get_context(article, span, mode=None):

  def words_count(sentence):
    return len(sentence.split(' '))

  if mode == "title":

    return article.split('\n')[0]
  if mode == "sentence":

    #setting the word limit to 150 which would get context upto 150 words.
    #Morethan this would make it too long.

    word_limit = 150
    start = span[0]
    end = span[1]
    span_text = article[start: end]
    word_count = words_count(span_text)
    if word_count >= word_limit:
      return span_text
    remaining_len = word_limit - word_count
    left_words = remaining_len // 2
    right_words = remaining_len - left_words
    try:
      if article[start-1]:

        start -= 1
    except:
      start=start
      pass
    l_word_count = 0
    print('indices are ',start,end)

    ##checking for both left hand and rihht hand side words
    while start >= 0  and l_word_count < left_words and article[start] != '\n':
      if article[start] == ' ':
        l_word_count += 1
      start -= 1
    end += 1


    r_word_count = 0
    while end < len(article) and article[end] != '\n' and r_word_count < right_words:
      if article[end] == ' ':
        r_word_count += 1
      end += 1

    return article[start+1: end - 1] 

  return ""

In [ ]:
def get_data_examples(articles, spans, techniques, context_mode=None):

  ##raise error if tehre is length mismatch
  assert len(articles) == len(spans) and len(spans) == len(techniques)
  labels,sentences,sent_contexts= [],[],[]

  for idx, arti in enumerate(articles):
    span = spans[idx]
    technique = techniques[idx]
    assert len(technique) == len(span)
    for i, sp in enumerate(span):
      pt = label2idx[technique[i]]
      sentence = arti[sp[0]: sp[1]]
      sentences.append(sentence)
      labels.append(pt)
      context = get_context(arti, sp, context_mode)
      sent_contexts.append(context)
  return sentences, labels, sent_contexts

In [ ]:


def convert_sentence_to_input_feature(sentence,
                                      tokenizer,
                                      add_cls_sep=True,
                                      max_seq_len=150, 
                                      context=None):
  


  #Get sentence encoded with adding the cls and sep tokens ..

  sentence_tokenized = tokenizer.encode_plus(sentence,
                                             add_special_tokens=add_cls_sep,
                                             pad_to_max_length=True,
                                             max_length=max_seq_len,
                                             return_attention_mask=True)
  

  ##the context obtained above is passed and encoded with adding cls ,sep tokens
  context_tokenized = tokenizer.encode_plus(context,
                                            add_special_tokens=add_cls_sep,
                                            max_length=max_seq_len,
                                            pad_to_max_length=True,
                                            return_attention_mask=True)
  
  return sentence_tokenized['input_ids'], sentence_tokenized['attention_mask'], context_tokenized['input_ids'], context_tokenized['attention_mask']

In [ ]:
def get_inputs_and_attention_masks(sentences):
  context_inputs,context_attention_masks = [],[]
  sent_inputs,sent_attention_masks = [],[]
  for i, sentence in enumerate(sentences):
    s_input_ids, s_mask, c_input_ids, c_mask = convert_sentence_to_input_feature(sentence, tokenizer, context=contexts[i])
    sent_inputs.append(s_input_ids)
    sent_attention_masks.append(s_mask)
    context_inputs.append(c_input_ids)
    context_attention_masks.append(c_mask)
  return sent_inputs,sent_attention,context_inputs,context_attention_masks

In [ ]:
def get_max_len(sentences):
  max_sent_len = 0
  for sent in sentences:
    sent_len = len(sent.split(' '))
    max_sent_len = max(max_sent_len, sent_len)
  return max_sent_len

In [ ]:

def get_data(articles, spans, techniques, context_mode=None):


  if context_mode is None:
    context_mode = C_mode #hyp param
    # context_mode = 'title' #hyp param


  sentences, labels, contexts = get_data_examples(articles, spans, techniques, context_mode=context_mode)
  sent_inputs,sent_attention,context_inputs,context_attention_masks=get_inputs_and_attention_masks(sentences)



##get max len for both sentencces and contexts 
  max_sent_len=get_max_len(sentences)
  max_context_len=get_max_len(contexts)
 
  print(max_sent_len, max_context_len)


  ##convert the list of input featueres into torch tensors to load into dataloader.
  sent_inputs = torch.tensor(sent_inputs)
  context_inputs = torch.tensor(context_inputs)
  labels = torch.tensor(labels)
  sent_masks = torch.tensor(sent_attention_masks)
  context_masks = torch.tensor(context_attention_masks)


  tensor_data_as_inp = TensorDataset(sent_inputs, labels, sent_masks, context_inputs, context_masks)
  dataloader = DataLoader(tensor_data_as_inp, batch_size=BATCH_SIZE) ##hyper param
  return dataloader

In [ ]:
from torch.nn import CrossEntropyLoss, MSELoss

class CustomModel(torch.nn.Module):
  
  def __init__(self, num_labels, ContextModel, SpanModel):
    super(CustomModel, self).__init__()

    self.ContextModel = ContextModel
    self.SpanModel = SpanModel
    self.num_labels = num_labels
    
    ##exp
    self.classifier_custom = torch.nn.Linear(768+128, num_labels)
    self.reduce_classifier_dims = torch.nn.Linear(768, 128)
    ##exp1
    self.dropout = torch.nn.Dropout(0.1)
    #exp2
    # self.dropout = torch.nn.Dropout(0.2)


  def forward(
      self,
      span_input_ids,
      span_attention_mask,
      context_input_ids,
      context_attention_mask,
      labels=None
  ):
    context_outputs = self.ContextModel(
        input_ids=context_input_ids,
        attention_mask=context_attention_mask
    )
    context_outputs = context_outputs[1] 
    span_outputs = self.SpanModel(
        input_ids=span_input_ids,
        attention_mask=span_attention_mask
    )
    span_outputs = span_outputs[1]

    context_outputs = self.reduce_classifier_dims(context_outputs)
    pooled_output =  self.dropout(torch.cat((span_outputs, context_outputs), axis=1))


    ##get the logit
    logits = self.classifier_custom(pooled_output)
    outputs = (logits,)

    
    if labels is not None:
      if self.num_labels == 1:
        loss_func = MSELoss()
        loss = loss_func(logits.view(-1), labels.view(-1))
      else:
        loss_func = CrossEntropyLoss()
        loss = loss_func(logits.view(-1, self.num_labels), labels.view(-1))
      outputs = (loss,) + outputs

    return outputs

In [ ]:
def calc_accuracy(preds, labels):
  preds = np.argmax(preds, axis=1).flatten()
  labels = labels.flatten()
  return np.sum(preds == labels) / len(labels)

In [ ]:
def train(model, epochs=5):
  loss_values = []
  for epoch_i in range(0, epochs):
    print('######### Epoch {:} / {:} #######'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
      if step % 100 == 0 and not step == 0:
        elapsed = time.time() - t0
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
      b_input_ids = batch[0].to(device)
      b_labels = batch[1].to(device)
      b_input_mask = batch[2].to(device)
      b_c_input_ids = batch[3].to(device)
      b_c_input_mask = batch[4].to(device)
      model.zero_grad()        
      outputs = model(b_input_ids, 
                      b_input_mask,
                      b_c_input_ids, 
                      b_c_input_mask, 
                      labels=b_labels)
      loss = outputs[0]
      total_loss += loss.item()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      scheduler.step()
    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)
    
    print("__________________")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(time.time() - t0))
    print("__________________")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for batch in eval_dataloader:
      # batch = tuple(t.to(device) for t in batch)
      b_input_ids = batch[0].to(device)
      b_labels = batch[1].to(device)
      b_input_mask = batch[2].to(device)
      b_c_input_ids = batch[3].to(device)
      b_c_input_mask = batch[4].to(device)
      with torch.no_grad():        
        outputs = model(b_input_ids, 
                        b_input_mask,
                        b_c_input_ids, 
                        b_c_input_mask)
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      tmp_eval_accuracy = calc_accuracy(logits, label_ids)
      eval_accuracy += tmp_eval_accuracy
      nb_eval_steps += 1

    print("---------------------------------------")
    print(" Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print(" Validation took: {:}".format(time.time() - t0))
    print("---------------------------------------")
  print("Training is complete!")

In [ ]:

#save the model TC
# model_name = 'model_' + str(datetime.datetime.now()) + '.pt'
# torch.save(model, os.path.join(model_dir, model_name))
# print("Model saved:", model_name)

In [ ]:
def get_model_predictions(model, dataloader):
  model.eval()
  predictions , true_labels = [], []
  nb_eval_steps = 0
  for batch in dataloader:
    b_input_ids = batch[0].to(device)
    b_labels = batch[1].to(device)
    b_input_mask = batch[2].to(device)
    b_c_input_ids = batch[3].to(device)
    b_c_input_mask = batch[4].to(device)
    with torch.no_grad():        
      logits = model(b_input_ids, 
                     b_input_mask,
                     b_c_input_ids, 
                     b_c_input_mask)
    logits = logits[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    pred_label = np.argmax(logits, axis=1)
    predictions.extend(pred_label)
    true_labels.extend(label_ids)
  return predictions, true_labels

In [ ]:
from google.colab import files

def get_dev_predictions(model):
  dev_articles, _,_ = read_articles("dev-articles")
  dev_spans, dev_techniques = read_dev_spans()

  dev_articles = dev_articles[1:]
  print('dev article count',len(dev_articles))
  print('dev spans len',len(dev_spans))
  dev_dataloader = get_data(dev_articles, dev_spans, dev_techniques)
  pred, _ = get_model_predictions(model, dev_dataloader)

  with open('predictions_tc.txt', 'w') as fp:
    label_file = os.path.join(data_dir, "dev-task-flc-tc.labels")
    myfile = open(label_file)
    prev_index = -1
    tsvreader = csv.reader(myfile, delimiter="\t")
    for i, row in enumerate(tsvreader):
      fp.write(row[0] + '\t' + all_techniques[pred[i]] + '\t' + row[2] + '\t' + row[3] + '\n')

In [ ]:
# file_ids=clean_test_data()

In [ ]:
from google.colab import files

from pathlib import Path

def get_test_predictions(model):
  test_articles,article_ids,article_fullnames = read_articles("test-articles")
  test_spans, test_techniques = read_test_spans()

  test_articles = test_articles[:]
  print('test article count',len(test_articles))
  print('test spans len',len(test_spans))
  test_dataloader = get_data(test_articles, test_spans, test_techniques)
  pred, _ = get_model_predictions(model, test_dataloader)

  with open('predictions_tc_test.txt', 'w') as fp:
    label_file = os.path.join(data_dir, "test-task-tc-template.out")
    myfile = open(label_file)
    prev_index = -1
    tsvreader = csv.reader(myfile, delimiter="\t")

    for i, row in enumerate(tsvreader):
      fp.write(row[0] + '\t' + all_techniques[pred[i]] + '\t' + row[2] + '\t' + row[3] + '\n')

In [ ]:
BATCH_SIZE=8
test_size = 0.1
seed_val = 2022

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import RobertaTokenizer

articles, article_ids,_ = read_articles("train-articles")
spans, techniques = read_spans()


all_techniques = list(set([y for x in techniques for y in x])) 

label2idx = {t: i for i, t in enumerate(all_techniques)}
print(label2idx)

num_artilces = len(articles)
articles = articles[0:num_artilces]
spans = spans[0:num_artilces]
techniques = techniques[0:num_artilces]

indices = np.arange(num_artilces)
train_articles, eval_articles, train_spans, eval_spans, train_techniques, eval_techniques, train_indices, eval_indices = train_test_split(articles, spans, techniques, indices, test_size=test_size)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', lower_case=True)

C_mode = "sentence" # sentence or title ::ps no good results with titile ..cntext not capturing properly


##loading train and eval loaders 
train_dataloader = get_data(train_articles, train_spans, train_techniques)
eval_dataloader = get_data(eval_articles, eval_spans, eval_techniques)

['article111111111.txt', 'article111111112.txt', 'article111111113.txt', 'article111111114.txt', 'article111111115.txt', 'article111111117.txt', 'article111111121.txt', 'article111111122.txt', 'article111111123.txt', 'article111111124.txt', 'article111111131.txt', 'article111111132.txt', 'article111111133.txt', 'article111111134.txt', 'article111111135.txt', 'article111111136.txt', 'article111111137.txt', 'article694327499.txt', 'article694356862.txt', 'article694811415.txt', 'article695108099.txt', 'article695833178.txt', 'article696246189.txt', 'article696264594.txt', 'article696694316.txt', 'article696735702.txt', 'article697063039.txt', 'article697444415.txt', 'article697454736.txt', 'article697472447.txt', 'article697959084.txt', 'article697996062.txt', 'article698018235.txt', 'article698092698.txt', 'article698503276.txt', 'article698719689.txt', 'article698780559.txt', 'article699142854.txt', 'article699291100.txt', 'article699478811.txt', 'article700387229.txt', 'article7004616

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
1616 1627
1631 1649
2842 2866
3611 3630
3910 3965
4001 4035
4094 4115
4130 4145
1298 1432
1433 1509
3952 3965
3978 4158
644 742
6850 6898
16494 16517
18425 18473
21885 21905
224 260
367 403
603 636
1793 1861
3537 3671
6928 6946
7132 7153
7730 7744
13377 13403
14277 14294
16986 17002
18428 18448
18737 18761
18931 18958
19545 19578
21374 21389
21537 21553
21765 21785
22023 22037
22773 22811
23320 23414
23699 23714
23756 23780
23822 23867
24442 24488
420 428
508 513
1089 1106
6045 6058
7200 7218
8305 8314
8906 8922
9193 9218
13404 13432
13499 13507
13996 14021
14626 14644
14808 14822
15613 15630
17740 17758
18365 18386
18655 18706
19053 19073
19376 19401
19465 19544
21303 21337
22302 22521
22718 22752
24897 24905
25126 25146
26580 26625
59 176
28 38
3014 3031
117 137
188 228
458 515
917 1208
1222 1265
2584 2611
2769 2829
3014 3033
3047 3065
3578 3602
3664 3744
3911 4039
516 629
318 349
917 1206
1222 1254
1798 1804
2701 2827
3487 3503
591 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


30248
30593 30612
30851 30872
30930 30967
30700 30900
31204 31218
31409 31438
31141 31280
31585 31656
31755 31781
31849 31873
31910 31946
32154 32196
32893 32959
33128 33156
34718 34815
35181 35214
35663 35814
35681 35739
35773 35814
35871 36008
36085 36449
36647 36684
37068 37101
37192 37232
37485 37576
37818 37855
37873 38077
38569 38665
38761 38802
39083 39201
39733 39790
39977 40056
39951 40189
40499 40629
40667 40693
40749 41110
41089 41110
41370 41386
41477 41496
41465 41543
41647 41664
41768 41788
41789 41826
42659 42701
43869 43891
44152 44273
46116 46145
46304 46328
46477 46554
46654 46673
46619 46760
46812 46852
46901 46948
47107 47126
47270 47306
47331 47391
6012 6234
6963 7010
8044 8094
10343 10372
24370 24386
25544 25559
29255 29266
29713 29724
31310 31349
31571 31946
31947 32272
39396 39470
39699 39919
41524 41536
45186 45317
2144 2414
2096 2131
2556 2582
107 116
280 302
2224 2268
2268 2414
2345 2365
33 58
258 282
319 336
516 535
685 703
1413 1439
2005 2036
2087 2117
5050

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


133 133
85 101
159 178
383 464
465 560
2508 2712
2536 2584
2595 2616
3634 3651
2945 3341
164 172
291 320
474 481
4686 4758
5551 5571
118 141
304 321
1754 1783
4047 4053
4148 4170
4270 4292
4520 4541
5613 5663
1970 2046
4390 4412
12172 12206
8016 8133
11404 11421
11939 12022
2748 2825
4177 4192
1742 1825
2450 2491
2631 2644
2631 2753
1372 1476
2855 3300
16 41
333 342
342 417
721 732
812 1045
1721 1740
2167 2191
3183 3203
3228 3250
2855 2965
60 82
1052 1367
1603 1724
410 417
1492 1512
847 1011
209 218
496 517
1380 1394
2027 2040
168 177
904 915
1168 1383
1331 1349
1384 1536
1537 1599
1864 2088
1234 1306
1456 1536
1918 1934
2027 2040
1894 1905
2885 2916
630 649
43 60
1013 1050
1128 1152
1266 1308
1467 1507
1472 1656
1763 1770
2933 2946
3165 3187
3207 3257
120 237
113 247
3741 3775
3857 4015
487 515
4524 4596
3593 3609
5318 5340
3474 3482
3694 3702
3654 3670
4061 4076
3795 3808
2662 2674
4195 4208
4237 4248
4252 4264
4331 4343
4430 4441
4545 4558
4640 4653
4717 4729
4776 4788
4858 4871
489

In [ ]:
##Experiment 1 
#(loading context and span models from bert base uncased with BIO labeling)
#tagging schema:BIO
#epochs:5
#LR scheduler 

##Experiment 2
#(loading context and span models from bert base uncased with BIOE labeling)
#tagging schema:BIOE
#epochs:5
#LR scheduler 


##Experiment 3 
#pretrained model:RoBERTa
#tagging_schema: BIOE
#Lr sheduler 
#epochs =5
#changing the linear layer size at the end and also adding dropout..


# The Roberta Model along with BIOE tagging scheme in the experiment 3 gave us best of all with F1 score of 0.574

In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = len(all_techniques)




#Experiment 1 (laoding context and span models from Roberta  )
context_model = RobertaModel.from_pretrained("roberta-base")
span_model = RobertaModel.from_pretrained("roberta-base")

#Experiment 2 (loading context and span models from Roberta with changein epochs and added LR scheduler below )


model = CustomModel(num_labels, context_model, span_model)
model.cuda()

optimizer = AdamW(model.parameters(),lr = 2e-5,eps = 2e-8) 
epochs = 4
total_steps = len(train_dataloader) * epochs

#Reference: from transformers module..
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
train(model, epochs=epochs)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaMod


======== Epoch 1 / 4 ========
Training...
  Batch   100  of    707.    Elapsed: 47.363404512405396.
  Batch   200  of    707.    Elapsed: 94.39915466308594.
  Batch   300  of    707.    Elapsed: 141.57959961891174.
  Batch   400  of    707.    Elapsed: 188.79348421096802.
  Batch   500  of    707.    Elapsed: 235.70703625679016.
  Batch   600  of    707.    Elapsed: 282.9736225605011.
  Batch   700  of    707.    Elapsed: 330.0960941314697.

  Average training loss: 1.73
  Training epcoh took: 333.379638671875

Running Validation...
 Accuracy: 0.46
 Validation took: 7.996367454528809

======== Epoch 2 / 4 ========
Training...
  Batch   100  of    707.    Elapsed: 47.19640016555786.
  Batch   200  of    707.    Elapsed: 94.47512483596802.
  Batch   300  of    707.    Elapsed: 141.67391991615295.
  Batch   400  of    707.    Elapsed: 188.80854773521423.
  Batch   500  of    707.    Elapsed: 235.7650215625763.
  Batch   600  of    707.    Elapsed: 282.7468752861023.
  Batch   700  of    

In [ ]:
##to save model

model_name = 'model_tc' + str(datetime.datetime.now()) + '.pt'
torch.save(model, os.path.join(model_dir, model_name))
print("Model saved:", model_name)


Model saved: model_tc2022-05-12 20:22:49.025565.pt


In [ ]:
# load saved model 

# model_test = torch.load(os.path.join(model_dir, 'model_tc2022-05-12 20:22:49.025565.pt')) # Test accuracy for custom model


In [ ]:
#to get test set predciitons for submission
get_test_predictions(model)

In [ ]:
##evaluate on dev set articles
get_dev_predictions(model)

['article730081389.txt', 'article730093263.txt', 'article730246508.txt', 'article730269378.txt', 'article738028498.txt', 'article738361208.txt', 'article738442776.txt', 'article738447109.txt', 'article738542398.txt', 'article738781754.txt', 'article740235127.txt', 'article761568202.txt', 'article761575506.txt', 'article761722669.txt', 'article761955563.txt', 'article763114850.txt', 'article763280007.txt', 'article763412406.txt', 'article772836731.txt', 'article773650987.txt', 'article776126299.txt', 'article776385494.txt', 'article777720051.txt', 'article777785889.txt', 'article777869943.txt', 'article778094905.txt', 'article778358096.txt', 'article778507244.txt', 'article778730964.txt', 'article779309765.txt', 'article781672902.txt', 'article782149032.txt', 'article782448403.txt', 'article783774960.txt', 'article784143418.txt', 'article784382409.txt', 'article785331076.txt', 'article786004130.txt', 'article787085939.txt', 'article787730392.txt', 'article787966255.txt', 'article7882714

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


80 98


In [ ]:
#downloading the file after running the get_predictions_test 

files.download('predictions_tc_test.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
##confusion matrix and Analysis

s, t = get_model_predictions(model_test, eval_dataloader)
print (len(s), len(t))
print(metrics.confusion_matrix(t, s))
print(metrics.classification_report(t, s))

472 472
[[  5   2   0   2   0   0   5   0   0   1   0   2   0   0]
 [  0   0   0   2   0   0   3   0   0   0   0   0   0   1]
 [  0   0   1   3   1   2   5   0   0   2   0   3   0   0]
 [  0   0   1  13   0   0   0   0   0   4   0   0   0   0]
 [  0   0   0   1  48   2   0  11   0   0   0   0   1   0]
 [  0   0   0   0   1  13   0  10   1   1   1   1   0   0]
 [  3   0   0   0   0   1  24   5   0   0   0   2   1   0]
 [  0   0   0   0   6  10   5 105   0   3   0   0   9   2]
 [  0   2   0   1   0   0   1   0   0   4   0   2   2   0]
 [  0   1   0   1   1   3   0   5   0  34   0   0   2   0]
 [  0   0   0   0   0   0   0   0   0   0   2   0   0   0]
 [  1   1   0   0   0   1   0   2   0   6   0   6   0   0]
 [  0   0   0   0   6   2   2  25   0   1   0   1  23   0]
 [  0   0   0   0   1   0   0   0   0   0   0   0   2   6]]
              precision    recall  f1-score   support

           0       0.56      0.29      0.38        17
           1       0.00      0.00      0.00         6
  